# Common Design Patterns

### Loading Libraries

In [1]:
# Math
import math
from math import hypot, factorial

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn
import matplotlib.pyplot as plt

#
from pprint import pprint

# OS
import io
import re
import sys
import abc
import csv
import time
import gzip
import queue
import heapq
import socket
import string
import random
import bisect
import operator
import datetime
import contextlib
import subprocess
from decimal import Decimal
from abc import ABC, abstractmethod

# Types & Annotations
import collections
from __future__ import annotations
from collections import defaultdict, Counter
from collections.abc import Container, Mapping, Hashable
from typing import TYPE_CHECKING
from typing import Pattern, Match
from typing import Hashable, Mapping, TypeVar, Any, overload, Union, Sequence, Dict, Deque, TextIO, Callable
from typing import List, Protocol, NoReturn, Union, Set, Tuple, Optional, Iterable, Iterator, cast, NamedTuple
# from typing import 

# Functional Tools
from functools import wraps, total_ordering, lru_cache

# Files & Path
import logging
import zipfile
import fnmatch
from pathlib import Path
from urllib.request import urlopen
from urllib.parse import urlparse

# Dataclass
from dataclasses import dataclass, field

## The Decorator Pattern

### A Decorator Example

In [2]:
def main_1() -> None:
    server = socket.socket(socket.AF_INET, socket.SOCKET_STREAM)
    server.bind(("localhost", 2401))
    with contextlib.closing(server):
        while True:
            client, addr = server.accept()
            dice_response(client)
            client.close()

In [3]:
def dice_response(client: socket.socket) -> None:
    request = client.recv(1024)
    try:
        response = dice.dice_roller(request)
    except (ValueError, KeyError) as ex:
        response = repr(ex).encode("utf-8")
    client.send(response)

In [4]:
def dice_roller(request: bytes) -> bytes:
    request_text = request.deconde("utf-8")
    numbers = [random.randint(1, 6) for _ in range(6)]
    response = f"{request_text} = {numbers}"
    return response.encode("utf-8")

In [5]:
# def main() -> None:
#     server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     server.connect(("localhost", 2401))
#     count = input("How many rolls: ") or "1"
#     pattern = input("Dice pattern nd6[dk+-]a: ") or "d6"
#     server.send(command.encode("utf-8"))
#     response = server.recv(1024)
#     print(response.decode("utf-8"))
#     server.close()

# if __name__ == "__main__":
#     main()

In [6]:
class LogSocket:
    def __init__(self, socket: socket.socket) -> None:
        self.socket = socket

    def recv(self, count: int = 0) ->bytes:
        data = self.socket.recv(count)
        print(f"Receiving {data!r} from {self.socket.getpeername()[0]})")
        return data

    def send(self, data: bytes) -> None:
        print(f"Sending {data!r} to {self.socket.getpeername()[0]}")
        self.socket.send(data)

    def close(self) -> None:
        self.socket.close()

In [7]:
def main_2() -> None:
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(("localhost", 2401))
    server.listen(1)
    with contextlib.closing(server):
        while True:
            client, addr = server.accept()
            logging_socket = cast(socket.scoket, LogSocket(client))
            dice_response(logging_socket)
            client.close()

In [8]:
Address = Tuple[str, int]

class LogRoller:
    def __init__(self, dice: Callable[[bytes], bytes], remote_addr: Address) -> None:
        self.dice_roller = dice
        self.remote_addr = remote_addr

    def __call__(self, request: bytes) -> bytes:
        print(f"Receiving {request!r} from {self.remote_addr}")
        dice_roller = self.dice_roller
        response = dice_roller(request)
        print(f"Sending {response!r} to {self.remote_addr}")
        return response

In [9]:
class ZipRoller:
    def __init__(self, dice: Callable[[bytes], bytes]) -> None:
        self.dice_roller = dice

    def __call__(self, request: bytes) -> bytes:
        dice_roller = self.deice_roller
        response = dice_roller(request)
        buffer = io.BytesIO()
        with gzip.GzipFile(fileobj=buffer, mode="w") as zipfile:
            zipfile.write(response)
        return buffer.getvalue()

In [10]:
def dice_response(client: socket.socket) -> None:
    request = client.recv(1024)
    try:
        remote_addr = client.getpeername()
        roller_1 = ZipRoller(dice.dice_roller)
        roller_2 = LogRoller(roller_1, remote_addr=remote_addr)
        response = roller_2(request)
    except (ValueError, KeyError) as ex:
        response = repr(ex).encode("utf-8")
    client.send(response)

In [11]:
# if config.zip_feature:
#     roller_1 = ZipRoller(dice.dice_roller)
# else:
#     roller_1 = dice.dice_roller

### Decorators in Python

In [12]:
def log_args(function: Callable[..., Any]) -> Callable[..., Any]:
    @wraps(function)
    def wrapped_function(*args: Any, **kwargs: Any) -> Any:
        print(f"Calling {function.__name__}(*{args}, **{kwargs})")
        result = function(*args, **kwargs)
        return result

    return wrapped_function

In [13]:
def trest1(a: int, b: int, c: int) ->float:
    return sum(range(a, b + 1)) / c

test1 = log_args(test1)

NameError: name 'test1' is not defined

In [14]:
# test1(1, 9, 2)

In [15]:
@log_args
def test1(a: int, b: int, c:int) -> float:
    return sum(range(a, b + 1)) / c

In [16]:
def binom(n: int, k: int) -> int:
    return factorial(n) // (factorial(k) * factorial(n-k))

In [17]:
f"6-card deals: {binom(52, 6):,d}"

'6-card deals: 20,358,520'

In [18]:
def binom(n: int, k: int) -> int:
    return factorial(n) // (factorial(k) * factorial(n-k))

In [19]:
class NamedLogger:
    def __init__(self, logger_name: str) -> None:
        self.logger = logging.getLogger(logger_name)

    def __call__(self, function: Callable[..., Any]) -> Callable[..., Any]:
        @wraps(function)
        def wrapped_function(*args: Any, **kwargs: Any) -> Any:
            start = time.perf_counter()
            try:
                result = function(*args, **kwargs)
                μs = (time.perf_counter() - start) * 1_000_000
                self.logger.info(f"{function.__name__}, {μs:.1f}μs")
                return result
            except Exception as ex:
                μs = (time.perf_counter() - start) * 1_000_000
                self.logger.error(f"{ex}, {function.__name__}, {μs:.1f}μs")
                raise

        return wrapped_function

## The Observer Pattern

### An Observer Example

In [20]:
class Observer(Protocol):
    def __call__(self) -> None:
        ...

class Observable:
    def __init__(self) -> None:
        self._observers: list[Observer] = []

    def attach(self, observer: Observer) -> None:
        self._observers.append(observer)

    def detach(self, observer: Observer) -> None:
        self._observers.remove(observer)

    def _notify_observers(self) -> None:
        for observer in self._observers:
            observer()

In [21]:
class ZonkHandHistory(Observable):
    def __init__(self, player: str, dice_set: Dice) -> None:
        super().__init__()
        self.player = player
        self.dice_set = dice_set
        self.rolls: list[Hand]

    def start(self) -> Hand:
        self.dice_set.roll()
        self.rolls = [self.dice_set.dice]
        self.notify_observers()
        return self.dice_set.dice

    def roll(self) -> Hand:
        self.dice_set.roll()
        self.rolls.append(self.dice_set.dice)
        self._notify_observers()
        return self.dice_set.dice

In [22]:
class SaveZonkHand(Observer):
    def __init__(self, hand: ZonkHandHistory) -> None:
        self.hand = hand
        self.count = 0

    def __call__(self) -> None:
        self.count += 1
        message = {
        "player": self.hand.player,
        "sequence": self.count,
        "hands": json.dumps(self.hand.rolls),
        "time": time.time()
        }
        print(f"SaveZonkHand {message}")

In [23]:
# d = Dice.from_text("6d6")

# palyer = ZonkHandHistory("Bo", d)

# save_history = SaveZonkHand(player)

In [24]:
class ThreePairZonkHand:
    """Observer of ZonkHandHistory"""
    def __init__(self, hand: ZonkHandHistory) -> None:
        self.hand = hand
        self.zonked = False

    def __call__(self) -> None:
        last_roll = set.hand.rolls[-1]
        distinct_values = set(last_roll)
        self.zonked = len(distinct_values) == 3 and all(last_roll.count(v) == 2 for v in distinct_values)
        if self.zonked:
            print("3 Pair Zonk!")

## The Strategy Pattern

### A Strategy Example

In [25]:
from PIL import Image #type: ignore [import]

In [26]:
Size = Tuple[int, int]

class FillAlgorithm(abc.ABC):
    @abc.abstractmethod
    def make_background(self, img_file: Path, desktop_size: Size) -> Image:
        pass

In [27]:
class TiledStrategy(FillAlgorithm):
    def make_background(self, img_file: Path, desktop_size: Size) -> Image:
        in_img = Image.open(img_file)
        out_img = image.new("RGB", desktop_size)
        num_tiles = [o // i + 1 for o, i in zip(out_img.size, in_img.size)]
        for x in range(num_tiles[0]):
            for y in range(num_tiles[1]):
                out_img.paste(in_img,
                              (
                                  in_img.size[0] * x,
                                  in_img.size[1] * y,
                                  in_img.size[0] * (x + 1),
                                  in_img.size[1] * (y + 1),
                              ),
                             )
                return out_img

In [30]:
class CenteredStrategy(FillAlgorithm):
    def make_background(self, img_file: Path, desktop_size: Size) -> Image:
        in_img = Image.open(img_file)
        out_img = Image.new("RGB", desktop_size)
        left = (out_img.size[0] - in_img.size[0]) // 2
        top = (out_img.size[1] - in_img.size[1]) // 2
        out_img.paste(in_img, 
                      (left, top, left + in_img.size[0], top + in_img.size[1]),
        )
        return out_img

In [31]:
class ScaledStrategy(FillAlgorithm):
    def make_background(self, img_file: Path, desktop_size: Size) -> Image:
        in_img = Image.open(img_file)
        out_img = in_img.resize(desktop_size)
        return out_img

In [32]:
class Resizer:
    def __init__(self, algorithm: FillAlgorithm) -> None:
        self.algorithm = algorithm

    def resize(self, image_file: Path, size: Size) -> Image:
        result = self.algorithm.make_background(image_file, size)
        return result

In [33]:
def main() -> None:
    image_file = Path.cwd() / "boat.png"
    tile_desktop = Resizer(TiledStragegy())
    title_image = tiled_desktop.resize(image_file, (1920, 1080))
    tiled_image.show()